In [1]:
import pandas as pd
import re
import pickle
from urllib.request import Request, urlopen
import bs4 as bs
import lxml
import json
import random
import pickle
import random
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import ftfy
import langdetect

#### Filter csv

In [2]:
df = pd.read_csv("../Data/filtered_df_muckrack_with_twitter8april2021.csv")

In [3]:
for i in range(len(df)):
    journos = json.loads(df.iloc[i].journalist)
    if len(journos) < 3:
        df.loc[i, "journalist"] = 'delete'

In [4]:
df = df[df.journalist != 'delete'].reset_index()

#### Get Alexa ranking

In [5]:
df.cleaned_outlet_url

0                  cbc.ca
1                  cbc.ca
2                  cbc.ca
3                  cbc.ca
4                  cbc.ca
              ...        
1100    opendemocracy.net
1101       nbcbayarea.com
1102      foxbusiness.com
1103              upi.com
1104         collider.com
Name: cleaned_outlet_url, Length: 1105, dtype: object

In [6]:
def get_alexa_ranking(url):
    try:
        xml = urlopen('http://data.alexa.com/data?cli=10&dat=s&url={}'.format(url)).read() 
        text = int(re.search(r'<POPULARITY[^>]*TEXT="(\d+)"', str(xml)).groups()[0])
    except:
        text = "Cannot find"
    return text

In [7]:
df['alexa_ranking'] = df.apply(lambda x: get_alexa_ranking(x['cleaned_outlet_url']), axis=1)

In [8]:
df = df[df.alexa_ranking != "Cannot find"].reset_index()

#### Get mean and median values

In [9]:
df = df[df.t_followers.notnull()]

In [10]:
def clean_followers(num):
    return int(num.replace(',', ''))

In [11]:
df['clean_t_followers'] = df.apply(lambda x: clean_followers(x['t_followers']), axis=1)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1022 entries, 0 to 1101
Data columns (total 45 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   level_0                      1022 non-null   int64 
 1   index                        1022 non-null   int64 
 2   Accepts contributed content  53 non-null     object
 3   Broadcast Affiliation        63 non-null     object
 4   Country                      1018 non-null   object
 5   Days Published               290 non-null    object
 6   Frequency                    481 non-null    object
 7   Instagram                    712 non-null    object
 8   Language                     1021 non-null   object
 9   Media Market                 220 non-null    object
 10  Pinterest                    329 non-null    object
 11  Radio Format                 90 non-null     object
 12  Radio Media Market           9 non-null      object
 13  Scope                        1022

In [13]:
df.clean_t_followers.mean()

1321046.7318982386

In [14]:
df.clean_t_followers.median()

196383.5

In [15]:
df.alexa_ranking.median()

4233.0

In [16]:
df.alexa_ranking.mean()

14643.293542074363

#### Cutoff by median

In [17]:
select_df = df[df.clean_t_followers >= df.clean_t_followers.median()]
select_df = select_df[select_df.alexa_ranking <= df.alexa_ranking.median()]

In [18]:
select_df = select_df.drop(columns=['level_0', 'index'])

In [19]:
select_df = select_df.reset_index()

In [20]:
select_df.muckrack_profile_url

0      https://muckrack.com/media-outlet/cbc-saskatch...
1        https://muckrack.com/media-outlet/cbc-toronto-8
2          https://muckrack.com/media-outlet/cbc_toronto
3           https://muckrack.com/media-outlet/cbc_ottawa
4            https://muckrack.com/media-outlet/cbcottawa
                             ...                        
278                https://muckrack.com/media-outlet/nfl
279            https://muckrack.com/media-outlet/infobae
280           https://muckrack.com/media-outlet/euronews
281                https://muckrack.com/media-outlet/nhk
282        https://muckrack.com/media-outlet/foxbusiness
Name: muckrack_profile_url, Length: 283, dtype: object

#### Pick up random rows + random authors

In [21]:
select_df

,index,Accepts contributed content,Broadcast Affiliation,Country,Days Published,Frequency,Instagram,Language,Media Market,Pinterest,...,t_profile_website,t_tweet,t_twitter_username,t_verified,twitter,url,youtube,multiple_url,alexa_ranking,clean_t_followers
0,0,NaN,NaN,Canada,NaN,NaN,https://instagram.com/cbc,English,NaN,NaN,...,http://www.cbcnews.ca,Protected Account,@CBCNews,Verified account,https://twitter.com/CBCNews,http://cbc.ca/news/canada/saskatchewan,https://youtube.com/user/CBCtv,y,1286,3078927
1,2,NaN,NaN,Canada,NaN,NaN,https://instagram.com/cbc,English,NaN,NaN,...,http://www.cbcnews.ca,Protected Account,@CBCNews,Verified account,https://twitter.com/CBCNews,http://cbc.ca/news/canada/toronto,https://youtube.com/user/CBCtv,y,1286,3078927
2,3,NaN,NaN,Canada,NaN,NaN,https://instagram.com/cbc,English,NaN,NaN,...,http://www.cbc.ca/toronto,Protected Account,@CBCToronto,Verified account,https://twitter.com/CBCToronto,http://cbc.ca/news/canada/toronto,https://youtube.com/user/CBCtv,y,1286,606229
3,6,NaN,NaN,Canada,NaN,NaN,https://www.instagram.com/cbcottawa/?hl=en,English,NaN,NaN,...,http://cbc.ca/ottawa,Protected Account,@CBCOttawa,Verified account,https://twitter.com/CBCOttawa,http://cbc.ca/news/canada/ottawa,https://youtube.com/user/CBCtv,y,1286,241485
4,10,NaN,NaN,Canada,NaN,Weekly,https://www.instagram.com/cbcottawa/,English,NaN,NaN,...,http://cbc.ca/ottawa,Protected Account,@CBCOttawa,Verified account,https://twitter.com/CBCOttawa,https://www.cbc.ca/news/canada/ottawa,NaN,n,1286,241485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,1070,NaN,NaN,United States of America,NaN,NaN,NaN,English,NaN,NaN,...,http://nfl.com/network,Protected Account,@nflnetwork,Verified account,https://twitter.com/nflnetwork,http://nfl.com,NaN,n,1400,4124366
279,1076,NaN,NaN,Argentina,"Mon, \r\r\nTue, \r\r\nWed, \r\r\nThu, \r\r\nFr...",Daily,https://www.instagram.com/infobaeamerica/,Spanish,NaN,NaN,...,http://infobae.com/america,Protected Account,@infobaeamerica,Verified account,https://twitter.com/InfobaeAmerica,http://infobae.com,https://www.youtube.com/c/infobae,n,720,1270539
280,1085,NaN,NaN,France,NaN,NaN,https://instagram.com/euronews.tv,"Arabic, \r\r\n \r\r\n ...",NaN,NaN,...,http://www.euronews.com/,Protected Account,@euronews,Verified account,https://twitter.com/euronews,http://euronews.com,http://www.youtube.com/euronews,n,2377,459247
281,1087,NaN,NaN,Japan,NaN,NaN,NaN,"English, \r\r\n \r\r\n ...",NaN,NaN,...,http://www6.nhk.or.jp/nhkpr/,Protected Account,@NHK_PR,Verified account,https://twitter.com/NHK_PR,http://nhk.or.jp/nhkworld/,https://www.youtube.com/user/NHKWorld,n,695,2141031


In [22]:
random_rows = [random.randint(0, len(select_df) - 1) for i in range(200)]
random_journos = []

In [23]:
for i in random_rows:
    journos = json.loads(select_df.iloc[i].journalist)
    journo = random.choice(journos)
    random_journos.append(journo)

In [24]:
len(random_journos)

200

#### Fetch articles

In [25]:
nltk.download('stopwords')
nltk.download('wordnet')
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
def clean(doc):
    doc = ftfy.fix_text(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","", normalized)
    y = processed.split()
    return ' '.join(y)

In [27]:
analyze_df = pd.DataFrame(columns = ['name', 'url', 'title', 'description', 'content', 'beat', 'predicted'])

In [28]:
for i in range(len(random_journos)):
    
    try:
    
        name = random_journos[i]['name']
        muckrack = random_journos[i]['profile_url'] + '/articles'

        req = Request(muckrack, headers = {'User-Agent': 'Mozilla/5.0'})
        sauce = urlopen(req).read()
        soup = bs.BeautifulSoup(sauce,'lxml')

        beat = soup.find(class_ = 'person-details-beats').find_all('a')
        beat = ", ".join([i.text for i in beat])

        articles = soup.find_all(class_ = "news-story")

        for article in articles:
            title = article.find(class_ = 'news-story-title').text
            description = article.find(class_ = 'news-story-body').text.split('—')[1].strip()
            if langdetect.detect(description) == 'en':
                content = clean(title * 4 + description * 2)

                # print(name, muckrack, title, description, content, beat)
                analyze_df = analyze_df.append({'name': name, 'url': muckrack, 'title': title, 'description': description, 
                                                'content': content, 'beat': beat, 'predicted': '-'}, ignore_index = True)
    
    except:
        pass

In [29]:
analyze_df

,name,url,title,description,content,beat,predicted
0,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,Dating Apps Are Joining In a Big Vaccine Push,Photo: Greg Nash (Getty Images)The White House...,dating apps joining big vaccine pushdating app...,U.S.,-
1,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,FCC Curbs Price-Gouging on Phone Calls for Inc...,Photo: Megan Jelinger (Getty Images)The FCC to...,fcc curb pricegouging phone call incarcerated ...,U.S.,-
2,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,TikTok Will Soon Let You Batch Delete Annoying...,Photo: Loic Venace (Getty Images)Too bad for y...,tiktok soon let batch delete annoying comments...,U.S.,-
3,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,TikTok Will Soon Let You Batch Delete Annoying...,"Photo: Loic Venace, Getty ImagesToo bad for yo...",tiktok soon let batch delete annoying comments...,U.S.,-
4,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,Crypto Investors Scheme to Gain Control of Tes...,The cryptocurrency market has taken a major di...,crypto investor scheme gain control tesla bitc...,U.S.,-
...,...,...,...,...,...,...,...
2648,"Alexander, Jasmine",https://muckrack.com/jasmine-alexander/articles,Choker at the House of Blues,After hearing about Choker from one of my clos...,choker house blueschoker house blueschoker hou...,Sports,-
2649,"Alexander, Jasmine",https://muckrack.com/jasmine-alexander/articles,21 Questions: Diana Pastora Carson,“I suppose the question “Tell us about yoursel...,question diana pastora carson question diana p...,Sports,-
2650,"Alexander, Jasmine",https://muckrack.com/jasmine-alexander/articles,21 Questions: Lizzy Havey,“My name is Elizabeth Rose Havey. But I usuall...,question lizzy havey question lizzy havey ques...,Sports,-
2651,"Alexander, Jasmine",https://muckrack.com/jasmine-alexander/articles,21 Questions: Kevin Almazan,Tell us about yourself. “I am at a point in my...,question kevin almazan question kevin almazan ...,Sports,-


#### Make predictions

In [30]:
iab_mod = pickle.load(open('../Data/IAB/IAB_classifier_2.p','rb'))
iab_vec = pickle.load(open('../Data/IAB/IAB_vectorizer_2.p','rb'))
iab_bin = pickle.load(open('../Data/IAB/IAB_binarizer_2.p','rb'))

In [31]:
def iab_taxonomy_v2(text):
    text = clean(text)
    text = iab_vec.transform([text])
    res = list(iab_bin.inverse_transform(iab_mod.predict(text))[0])
    if '-' in res:
        res.remove('-')
    return res

In [32]:
analyze_df['predicted'] = analyze_df.apply(lambda x: iab_taxonomy_v2(x['content']), axis = 1)

In [33]:
analyze_df

,name,url,title,description,content,beat,predicted
0,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,Dating Apps Are Joining In a Big Vaccine Push,Photo: Greg Nash (Getty Images)The White House...,dating apps joining big vaccine pushdating app...,U.S.,"[Dating, Family and Relationships]"
1,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,FCC Curbs Price-Gouging on Phone Calls for Inc...,Photo: Megan Jelinger (Getty Images)The FCC to...,fcc curb pricegouging phone call incarcerated ...,U.S.,[]
2,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,TikTok Will Soon Let You Batch Delete Annoying...,Photo: Loic Venace (Getty Images)Too bad for y...,tiktok soon let batch delete annoying comments...,U.S.,[]
3,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,TikTok Will Soon Let You Batch Delete Annoying...,"Photo: Loic Venace, Getty ImagesToo bad for yo...",tiktok soon let batch delete annoying comments...,U.S.,[]
4,"Kimball, Whitney",https://muckrack.com/whitney-kimball/articles,Crypto Investors Scheme to Gain Control of Tes...,The cryptocurrency market has taken a major di...,crypto investor scheme gain control tesla bitc...,U.S.,"[Hedge Funds, Personal Finance, Personal Inves..."
...,...,...,...,...,...,...,...
2648,"Alexander, Jasmine",https://muckrack.com/jasmine-alexander/articles,Choker at the House of Blues,After hearing about Choker from one of my clos...,choker house blueschoker house blueschoker hou...,Sports,[]
2649,"Alexander, Jasmine",https://muckrack.com/jasmine-alexander/articles,21 Questions: Diana Pastora Carson,“I suppose the question “Tell us about yoursel...,question diana pastora carson question diana p...,Sports,"[Content Source Geo, Country]"
2650,"Alexander, Jasmine",https://muckrack.com/jasmine-alexander/articles,21 Questions: Lizzy Havey,“My name is Elizabeth Rose Havey. But I usuall...,question lizzy havey question lizzy havey ques...,Sports,[]
2651,"Alexander, Jasmine",https://muckrack.com/jasmine-alexander/articles,21 Questions: Kevin Almazan,Tell us about yourself. “I am at a point in my...,question kevin almazan question kevin almazan ...,Sports,[]


In [34]:
analyze_df.to_csv('../Data/double_randomized_results3.csv')